In [1]:
import numpy as np
import pandas as pd
import gc

In [2]:
print("Read data file to obtain some statistic about the features and target")
data = pd.read_csv('clean_data_tree.csv')
print "shape of data file: ", data.shape

Read data file to obtain some statistic about the features and target
shape of data file:  (35604, 24)


In [3]:
import xgboost as xgb
from sklearn.metrics import log_loss

class XGBoostClass():
    def __init__(self, num_boost_round=10, **kwargs):
        self.clf = None
        self.num_boost_round = num_boost_round
        self.params = kwargs
        self.params.update({'objective': 'multi:softprob'})
 
    def fit(self, X, y, num_boost_round=None):
        num_boost_round = num_boost_round or self.num_boost_round
        dtrain = xgb.DMatrix(X, label=y)
        self.clf = xgb.train(params=self.params, dtrain=dtrain, num_boost_round=num_boost_round)
 
    def predict(self, X):
        dtest = xgb.DMatrix(X)
        return self.clf.predict(dtest)
 
    def score(self, X, y):
        Y = self.predict(X)
        return r2_score(y, Y)
 
    def get_params(self, deep=True):
        return self.params
 
    def set_params(self, **params):
        if 'num_boost_round' in params:
            self.num_boost_round = params.pop('num_boost_round')
        if 'objective' in params:
            del params['objective']
        self.params.update(params)
        return self

C:\Users\Amin\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
model = XGBoostClass(num_class = 2, nround = 50)

In [5]:
train = data.drop("target", axis=1)
target = data["target"]

In [6]:
from sklearn.model_selection import KFold
skf = KFold(n_splits = 10, shuffle = True, random_state = 44)

scores = []
for train_index, test_index in skf.split(train):
        x0, x1 = train.iloc[train_index], train.iloc[test_index]
        y0, y1 = target.iloc[train_index], target.iloc[test_index] 
        model.fit(x0, y0)            
        predict = model.predict(x1)
        scores.append(log_loss(y1, predict))
        print log_loss(y1, predict)
        
print "mean logloss score in 10 folds: ", np.mean(scores), np.std(scores)

0.355203122749
0.347069938553
0.354741107222
0.368206980763
0.355506957848
0.367859146151
0.347352054139
0.351875386968
0.342725291124
0.351145876602
mean logloss score in 10 folds:  0.354168586212 0.00793660938007
